# Work in progress/Tests on the whole Telegram posting thing

In [ ]:
!python3.10 -m pip install Flask
!python3.10 -m pip install asyncio

In [ ]:
#!python3.10 -m pip install python-telegram-bot
#!python3.10 -m pip install Flask
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path(".env")
load_dotenv(dotenv_path=dotenv_path)
telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN")

from telegram.ext import Application, CommandHandler
from telegram import InputMediaPhoto

# from flask import Flask, request
import asyncio


import logging
import time

# Set up logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)

# Connect to db
db = dict(subscribers=list())

# Rate limiting
MESSAGE_LIMIT = 20
TIME_WINDOW = 60  # seconds
last_messages = []


def can_send_message():
    current_time = time.time()
    # Remove messages outside the time window
    global last_messages
    last_messages = [
        msg_time for msg_time in last_messages if current_time - msg_time < TIME_WINDOW
    ]
    return len(last_messages) < MESSAGE_LIMIT


def record_message_sent():
    last_messages.append(time.time())


async def start(update, context):
    await sub(update, context)


async def sub(update, context):
    chat_id = update.effective_chat.id
    db["subscribers"].append(chat_id) if chat_id not in db["subscribers"] else None
    sub_text = """
    You are now subscribed to these news.
    To unsubscribe, write /unsub. To subscribe again, write /sub.
    """
    await context.bot.send_message(chat_id=chat_id, text=sub_text)


async def unsub(update, context):
    chat_id = update.effective_chat.id
    db["subscribers"].remove(chat_id) if chat_id in db["subscribers"] else None
    unsub_text = """
    You are now unsubscribed from these news.
    To subscribe again, write /sub.
    """
    await context.bot.send_message(chat_id=chat_id, text=unsub_text)


async def get_chat_id(update, context):
    chat_id = update.effective_chat.id
    await context.bot.send_message(chat_id=chat_id, text=chat_id)


async def send_album_to_all_subscribers(image_paths, caption):
    subscriber_ids = db["subscribers"]
    for chat_id in subscriber_ids:
        if can_send_message():
            try:
                send_album(chat_id, image_paths, caption)
                record_message_sent()
            except Exception as e:
                logging.error(e)
        else:
            logging.info("Rate limit exceeded, waiting...")
            time.sleep(TIME_WINDOW - (time.time() - last_messages[0]))
            try:
                send_album(chat_id, image_paths, caption)
                record_message_sent()
            except Exception as e:
                logging.error(e)


async def send_album(chat_id, image_paths, caption):

    media_list = [
        InputMediaPhoto(media=open(image_path, "rb"), caption=caption)
        for image_path in image_paths
    ]

    # album = InputMediaPhoto(media=media_list, caption=caption)
    await application.bot.send_media_group(
        chat_id=chat_id, media=media_list
    )  # [album])


# app = Flask(__name__)
# app.run()
# @app.route('/' + "send_album", methods=['POST'])
# def webhook():
#     data = request.get_json()
#     send_album(**data)
#     return 'OK'


async def send_album_command(update, context):
    chat_id = update.effective_chat.id
    image_paths = ["tests/images/0.jpg", "tests/images/1.jpg"]
    caption = "[bla] [blub]"
    # loop = asyncio.get_event_loop()
    # loop.run_until_complete(send_album(chat_id, image_paths, caption))
    await send_album(chat_id, image_paths, caption)


# async def main():
#     print(1)


# Example of sending a message from Python code
if __name__ == "__main__":
    print("hi")
    # Initialize the bot
    application = Application.builder().token(telegram_bot_token).build()
    # send_album_to_all_subscribers("Hello, this is a test message from Python!")
    # Add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("sub", sub))
    application.add_handler(CommandHandler("unsub", unsub))
    application.add_handler(CommandHandler("get_chat_id", get_chat_id))
    application.add_handler(CommandHandler("send_album", send_album_command))
    await application.initialize()  # inits bot, update, persistence
    await application.start()
    await application.updater.start_polling()


# await main()
# asyncio.run(send_album(chat_id, image_paths, caption))


# application.updater.start_webhook(listen="0.0.0.0", port=int(os.environ.get('PORT', '8080')), url_path="was_treibt_der_bundestag_bot")

# async def main():
#     await application.run_polling()

# if __name__ == '__main__':
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(main())

#     # Start the bot
#     #application.wait_for_shutdown()